In [1]:
import pandas as pd
import numpy as np 
from tqdm import tqdm 
tqdm.pandas()
data = pd.read_excel("bagic_file.xlsx",skiprows=1)
pd.options.display.max_colwidth = 1000
data.drop(columns=['Idx','pdf','Flag1','Flag2','Flag3'],inplace=True)
data.columns = [s.replace(" ", "_") for s in list(data.columns)]

# 3, 2, 4, 6, 1, 5

In [2]:
rules = pd.DataFrame({'rules':['Meta_Flags','Colour_Anomalies','Missing_Stamp','Missing_Sign_on_Stamp','Image_Tampering_-_High_Risk','Image_Tampering - Medium_Risk'],'Factor':[3,2,4,6,1,5]})
dicts  = dict(zip(rules['rules'], rules['Factor']))

In [3]:
data['columns_list'] = [data.iloc[:, 5:].columns.tolist()]*len(data)
data['lists'] = data.iloc[:, 5:10].apply(lambda x: list(x), axis =1)
data['lists2'] = data.apply(lambda x: [i for i,j in enumerate(x['lists']) if j !=0], axis =1)
data['list3'] = data.apply(lambda x: [j for i,j in enumerate(x['columns_list']) if i in x['lists2']], axis=1)
data.head(2)

,ClaimId,Type,pageNo,imageType,Image_Risk,Meta_Flags,Colour_Anomalies,Missing_Stamp,Missing_Sign_on_Stamp,Image_Tampering_-_High_Risk,Image_Tampering_-_Medium_Risk,columns_list,lists,lists2,list3
0,OC-22-1002-6041-00005480,Invoice,out8.jpeg,Digital,Medium,1,0,0,1,0,1,"[Meta_Flags, Colour_Anomalies, Missing_Stamp, Missing_Sign_on_Stamp, Image_Tampering_-_High_Risk, Image_Tampering_-_Medium_Risk]","[1, 0, 0, 1, 0]","[0, 3]","[Meta_Flags, Missing_Sign_on_Stamp]"
1,210013593P,Invoice,out7.jpeg,Digital,Medium,1,0,1,0,0,0,"[Meta_Flags, Colour_Anomalies, Missing_Stamp, Missing_Sign_on_Stamp, Image_Tampering_-_High_Risk, Image_Tampering_-_Medium_Risk]","[1, 0, 1, 0, 0]","[0, 2]","[Meta_Flags, Missing_Stamp]"


In [4]:
x = data['ClaimId	Type	pageNo	imageType	Image_Risk list3'.split()].explode("list3").groupby('ClaimId	Type	pageNo	imageType	Image_Risk'.split())["list3"].agg(pd.Series.mode).to_frame()
x['dicts'] = [dicts]*len(x)
# x['dicts2'] = x.apply(lambda x: {k:v for k,v in (x['dicts'].items()) if k in x['list3']}, axis =1)

### this will sort value on the basis of the key  not their values :->

x['dicts2'] = x.apply(lambda x: {k: x['dicts'][k] for k in sorted(x['dicts']) if k in x['list3']}, axis=1)
x['len_dicts'] = x['dicts2'].apply(lambda x: len(x.keys()))

# x.drop("list3	dicts".split(), axis =1, inplace = True)

In [5]:
x.head(1)

,,,,,list3,dicts,dicts2,len_dicts
ClaimId,Type,pageNo,imageType,Image_Risk,,,,
210000083P,Discharge_summary,out1.jpeg,Digital,Medium,"[Meta_Flags, Missing_Stamp]","{'Meta_Flags': 3, 'Colour_Anomalies': 2, 'Missing_Stamp': 4, 'Missing_Sign_on_Stamp': 6, 'Image_Tampering_-_High_Risk': 1, 'Image_Tampering - Medium_Risk': 5}","{'Meta_Flags': 3, 'Missing_Stamp': 4}",2


In [6]:
x['Flag1'] = x['dicts2'].apply(lambda x: list(x.keys())[0] if isinstance(x, dict) and len(x) > 0 else np.NaN)
x['Flag2'] = x['dicts2'].apply(lambda x: list(x.keys())[1] if isinstance(x, dict) and len(x) > 1 else np.NaN)
x['Flag3'] = x['dicts2'].apply(lambda x: list(x.keys())[2] if isinstance(x, dict) and len(x) > 2 else np.NaN)


In [7]:
# x[x.dicts3==0]
x.head()

list3  \
ClaimId    Type              pageNo    imageType Image_Risk                                
210000083P Discharge_summary out1.jpeg Digital   Medium      [Meta_Flags, Missing_Stamp]   
                             out2.jpeg Digital   Medium      [Meta_Flags, Missing_Stamp]   
                             out3.jpeg Digital   Medium                       Meta_Flags   
           Invoice           out1.jpeg Digital   Medium                       Meta_Flags   
                             out2.jpeg Digital   Medium      [Meta_Flags, Missing_Stamp]   

                                                                                                                                                                                                                      dicts  \
ClaimId    Type              pageNo    imageType Image_Risk                                                                                                                                                                   
210000083P Discharge_summary out1.jpeg Digital   Medium      {'Meta_Flags': 3, 'Colour_Anomalies': 2, 'Missing_Stamp': 4, 'Missing_Sign_on_Stamp': 6, 'Image_Tampering_-_High_Risk': 1, 'Image_Tampering - Medium_Risk': 5}   
                             out2.jpeg Digital   Medium      {'Meta_Flags': 3, 'Colour_Anomalies': 2, 'Missing_Stamp': 4, 'Missing_Sign_on_Stamp': 6, 'Image_Tampering_-_High_Risk': 1, 'Image_Tampering - Medium_Risk': 5}   
                             out3.jpeg Digital   Medium      {'Meta_Flags': 3, 'Colour_Anomalies': 2, 'Missing_Stamp': 4, 'Missing_Sign_on_Stamp': 6, 'Image_Tampering_-_High_Risk': 1, 'Image_Tampering - Medium_Risk': 5}   
           Invoice           out1.jpeg Digital   Medium      {'Meta_Flags': 3, 'Colour_Anomalies': 2, 'Missing_Stamp': 4, 'Missing_Sign_on_Stamp': 6, 'Image_Tampering_-_High_Risk': 1, 'Image_Tampering - Medium_Risk': 5}   
                             out2.jpeg Digital   Medium      {'Meta_Flags': 3, 'Colour_Anomalies': 2, 'Missing_Stamp': 4, 'Missing_Sign_on_Stamp': 6, 'Image_Tampering_-_High_Risk': 1, 'Image_Tampering - Medium_Risk': 5}   

                                                                                            dicts2  \
ClaimId    Type              pageNo    imageType Image_Risk                                          
210000083P Discharge_summary out1.jpeg Digital   Medium      {'Meta_Flags': 3, 'Missing_Stamp': 4}   
                             out2.jpeg Digital   Medium      {'Meta_Flags': 3, 'Missing_Stamp': 4}   
                             out3.jpeg Digital   Medium                          {'Meta_Flags': 3}   
           Invoice           out1.jpeg Digital   Medium                          {'Meta_Flags': 3}   
                             out2.jpeg Digital   Medium      {'Meta_Flags': 3, 'Missing_Stamp': 4}   

                                                             len_dicts  \
ClaimId    Type              pageNo    imageType Image_Risk              
210000083P Discharge_summary out1.jpeg Digital   Medium              2   
                             out2.jpeg Digital   Medium              2   
                             out3.jpeg Digital   Medium              1   
           Invoice           out1.jpeg Digital   Medium              1   
                             out2.jpeg Digital   Medium              2   

                                                                  Flag1  \
ClaimId    Type              pageNo    imageType Image_Risk               
210000083P Discharge_summary out1.jpeg Digital   Medium      Meta_Flags   
                             out2.jpeg Digital   Medium      Meta_Flags   
                             out3.jpeg Digital   Medium      Meta_Flags   
           Invoice           out1.jpeg Digital   Medium      Meta_Flags   
                             out2.jpeg Digital   Medium      Meta_Flags   

                                                                     Flag2  \
ClaimId    Type              pageNo   

In [8]:
data[(data['ClaimId']=='210000083P')&(data['Type']=='Discharge_summary')&(data['pageNo']=='out1.jpeg')]

,ClaimId,Type,pageNo,imageType,Image_Risk,Meta_Flags,Colour_Anomalies,Missing_Stamp,Missing_Sign_on_Stamp,Image_Tampering_-_High_Risk,Image_Tampering_-_Medium_Risk,columns_list,lists,lists2,list3
200,210000083P,Discharge_summary,out1.jpeg,Digital,Medium,1,0,1,0,0,0,"[Meta_Flags, Colour_Anomalies, Missing_Stamp, Missing_Sign_on_Stamp, Image_Tampering_-_High_Risk, Image_Tampering_-_Medium_Risk]","[1, 0, 1, 0, 0]","[0, 2]","[Meta_Flags, Missing_Stamp]"


### Now using An Another Method

In [9]:
data = pd.read_excel("bagic_file.xlsx",skiprows=1)
pd.options.display.max_colwidth = 1000
data.drop(columns=['Idx','pdf','Flag1','Flag2','Flag3'],inplace=True)
data.columns = [s.replace(" ", "_") for s in list(data.columns)]
rules = pd.DataFrame({'rules':['Meta_Flags','Colour_Anomalies','Missing_Stamp','Missing_Sign_on_Stamp','Image_Tampering_-_High_Risk','Image_Tampering_-_Medium_Risk'],'Factor':[3,2,4,6,1,5]})
rules

,rules,Factor
0,Meta_Flags,3
1,Colour_Anomalies,2
2,Missing_Stamp,4
3,Missing_Sign_on_Stamp,6
4,Image_Tampering_-_High_Risk,1
5,Image_Tampering_-_Medium_Risk,5


In [10]:
dicts = dict(zip(rules['rules'], rules['Factor']))

In [11]:
data['list_of_all_flags'] = data.iloc[:, 5:].apply(lambda x: [i for i, j in enumerate (list(x)) if j!=0] , axis =1)
data['list_of_all_columns'] = [data.iloc[:, 5:].columns.to_list()]*len(data)
data['list_of_all_columns'] = data.apply(lambda x: [j for i,j in enumerate(x['list_of_all_columns']) if i in (x['list_of_all_flags'])], axis = 1)

In [12]:
data['rules_dict'] = [dicts]*len(data)
data['rules_dict'] = data.apply(lambda x: {k:v for k,v in (x['rules_dict'].items()) if k in x['list_of_all_columns']}, axis =1)

### creating function that will sort the dictionary by their values :->

def sort_dict_by_value(d):
    sorted_dict = {k: v for k, v in sorted(d.items(), key=lambda item: item[1])}
    return sorted_dict

data['rules_dict'] = data['rules_dict'].apply(lambda x: sort_dict_by_value(x))

data['Flag1'] = data['rules_dict'].apply(lambda x: list(x.keys())[0] if isinstance(x, dict) and len(x) > 0 else np.NaN)
data['Flag2'] = data['rules_dict'].apply(lambda x: list(x.keys())[1] if isinstance(x, dict) and len(x) > 1 else np.NaN)
data['Flag3'] = data['rules_dict'].apply(lambda x: list(x.keys())[2] if isinstance(x, dict) and len(x) > 2 else np.NaN)


In [13]:
data.head(10)

,ClaimId,Type,pageNo,imageType,Image_Risk,Meta_Flags,Colour_Anomalies,Missing_Stamp,Missing_Sign_on_Stamp,Image_Tampering_-_High_Risk,Image_Tampering_-_Medium_Risk,list_of_all_flags,list_of_all_columns,rules_dict,Flag1,Flag2,Flag3
0,OC-22-1002-6041-00005480,Invoice,out8.jpeg,Digital,Medium,1,0,0,1,0,1,"[0, 3, 5]","[Meta_Flags, Missing_Sign_on_Stamp, Image_Tampering_-_Medium_Risk]","{'Meta_Flags': 3, 'Image_Tampering_-_Medium_Risk': 5, 'Missing_Sign_on_Stamp': 6}",Meta_Flags,Image_Tampering_-_Medium_Risk,Missing_Sign_on_Stamp
1,210013593P,Invoice,out7.jpeg,Digital,Medium,1,0,1,0,0,0,"[0, 2]","[Meta_Flags, Missing_Stamp]","{'Meta_Flags': 3, 'Missing_Stamp': 4}",Meta_Flags,Missing_Stamp,NaN
2,OC-22-1002-6038-00003529,Invoice,out1.jpeg,Digital,Low,0,0,0,0,0,0,[],[],{},NaN,NaN,NaN
3,OC-22-1002-6041-00006051,Discharge_summary,out2.jpeg,Digital,Medium,0,0,1,0,0,1,"[2, 5]","[Missing_Stamp, Image_Tampering_-_Medium_Risk]","{'Missing_Stamp': 4, 'Image_Tampering_-_Medium_Risk': 5}",Missing_Stamp,Image_Tampering_-_Medium_Risk,NaN
4,OC-22-1002-6041-00005479,Discharge_summary,out2.jpeg,HW,High,0,1,0,1,0,1,"[1, 3, 5]","[Colour_Anomalies, Missing_Sign_on_Stamp, Image_Tampering_-_Medium_Risk]","{'Colour_Anomalies': 2, 'Image_Tampering_-_Medium_Risk': 5, 'Missing_Sign_on_Stamp': 6}",Colour_Anomalies,Image_Tampering_-_Medium_Risk,Missing_Sign_on_Stamp
5,OC-22-1002-6041-00005050,Invoice,out7.jpeg,Digital,Low,0,0,0,1,0,0,[3],[Missing_Sign_on_Stamp],{'Missing_Sign_on_Stamp': 6},Missing_Sign_on_Stamp,NaN,NaN
6,OC-22-1002-6041-00005030,Invoice,out6.jpeg,Digital,Medium,0,0,0,1,0,1,"[3, 5]","[Missing_Sign_on_Stamp, Image_Tampering_-_Medium_Risk]","{'Image_Tampering_-_Medium_Risk': 5, 'Missing_Sign_on_Stamp': 6}",Image_Tampering_-_Medium_Risk,Missing_Sign_on_Stamp,NaN
7,OC-22-1002-6041-00005492,Invoice,out3.jpeg,Digital,Medium,0,0,0,1,0,1,"[3, 5]","[Missing_Sign_on_Stamp, Image_Tampering_-_Medium_Risk]","{'Image_Tampering_-_Medium_Risk': 5, 'Missing_Sign_on_Stamp': 6}",Image_Tampering_-_Medium_Risk,Missing_Sign_on_Stamp,NaN
8,OC-22-1002-6041-00005325,Invoice,out3.jpeg,Digital,Medium,0,0,1,0,0,1,"[2, 5]","[Missing_Stamp, Image_Tampering_-_Medium_Risk]","{'Missing_Stamp': 4, 'Image_Tampering_-_Medium_Risk': 5}",Missing_Stamp,Image_Tampering_-_Medium_Risk,NaN
9,OC-22-1002-6041-00006051,Discharge_summary,out1.jpeg,Digital,Medium,0,0,1,0,0,1,"[2, 5]","[Missing_Stamp, Image_Tampering_-_Medium_Risk]","{'Missing_Stamp': 4, 'Image_Tampering_-_Medium_Risk': 5}",Missing_Stamp,Image_Tampering_-_Medium_Risk,NaN
